In [ ]:
#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## OUTLIER DETECTION DAY 3/OUTLIER DETECTION PART 5 ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs




In [ ]:
#=================================================-
#### Slide 4: Import packages  ####

import os
from pathlib import Path
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error




In [ ]:
#=================================================-
#### Slide 5: Directory settings  ####

# Set 'main_dir' to location of the project folder
from pathlib import Path 
home_dir = Path(".").resolve()
main_dir = home_dir.parent
print(main_dir)
data_dir = str(main_dir) + "/data"
print(data_dir)




In [ ]:
#=================================================-
#### Slide 6: Load PJM Energy dataset  ####

pjm_energy = pd.read_csv(str(data_dir) + "/PJME_hourly.csv")
print(pjm_energy.head())
pjm_energy['Datetime'] = pd.to_datetime(pjm_energy['Datetime'])
pjm_energy.dtypes




In [ ]:
#=================================================-
#### Slide 7: Extract Date from Datetime  ####

pjm_energy = pjm_energy.sort_values(by=['Datetime'])
pjm_energy = pjm_energy[pjm_energy['Datetime'] > '2018-01-01 00:00:00']
pjm_energy.shape
pjm_energy['Date'] = pjm_energy['Datetime'].dt.date
pjm_energy_datewise = pjm_energy.groupby(by=['Date'])['PJME_MW'].mean()




In [ ]:
#=================================================-
#### Slide 8: Visualize the Energy consumption for each day  ####

pjm_energy_datewise.plot(x='Date', y='PJME_MW', figsize=(17,6))
plt.xlabel('Date')
plt.ylabel('Energy Consumption')
plt.title('Energy consumption (MW) at each day in 2018')
plt.show()




In [ ]:
#=================================================-
#### Slide 16: Stationarity test  ####

# Perform ADF test on original series.
result_pm = adfuller(pjm_energy_datewise)

print('ADF Statistic: %f' % result_pm[0])
print('p-value: %f' % result_pm[1])




In [ ]:
#=================================================-
#### Slide 18: Fitting ARIMA model  ####

# Set up ARIMA model using train data only.
arima = ARIMA(pjm_energy_datewise, order = (5, 1, 0))
# Fit ARIMA model to train data only.
arima_fit = arima.fit()




In [ ]:
#=================================================-
#### Slide 19: Forecast vs. Actual Plot   ####

# Fitted values.
predicted = arima_fit.predict(typ='levels')
fig, ax = plt.subplots(figsize = (15, 4))
ax.plot(pjm_energy_datewise, linestyle = '-', linewidth = 1.5, label = 'Actual value')
ax.plot(predicted, markersize = 5, linewidth = 2.5, linestyle = '-',label = 'Forecast value')
ax.set_xlabel('Date', fontsize = 14)
ax.set_title("Actual vs forecast values for ARIMA(5, 1, 0)", fontsize = 20)
ax.legend(loc='best', fontsize=12)




In [ ]:
#=================================================-
#### Slide 22: Calculate error and static threshold  ####

results = pd.DataFrame(pjm_energy_datewise)
results['predicted'] = predicted
results['error'] = abs(results['PJME_MW'] - results['predicted'])
results['static_threshold'] = np.mean(results['error']) + 3 * np.std(results['error'])
results['static_anomaly'] = results['error'] >= results['static_threshold']
results.head()




In [ ]:
#=================================================-
#### Slide 23: Anomalies detected with static threshold  ####

results['static_anomaly'].value_counts()
results[results['static_anomaly'] == True]




In [ ]:
#=================================================-
#### Slide 24: Visualize the anomalies  ####

fig, ax = plt.subplots(figsize = (15, 4))
ax.plot(results['PJME_MW'],markersize = 5, linewidth = 2.5, linestyle = '-', 
        label = 'Actual value')
ax.scatter(results[results['static_anomaly'] == True].index,results[results['static_anomaly'] == True]['PJME_MW'], 
        color='red',label = 'Anomaly')
ax.set_xlabel('Date', fontsize = 14)
ax.set_title("Static anomalies", fontsize = 20)
ax.legend()
plt.show()




In [ ]:
#=================================================-
#### Slide 25: Calculate dynamic threshold  ####

results['error_windowed_mean'] = results['error'].rolling(window=7, min_periods=1).mean()
results['error_windowed_std'] = results['error'].rolling(window=7, min_periods=1).std()
results['dynamic_threshold'] = results['error_windowed_mean'] + (2*results['error_windowed_std'])
results['dynamic_anomaly'] = results['error'] >= results['dynamic_threshold']




In [ ]:
#=================================================-
#### Slide 26: Anomalies detected with dynamic threshold  ####

results['dynamic_anomaly'].value_counts()
results[results['dynamic_anomaly'] == True]




In [ ]:
#=================================================-
#### Slide 27: Visualize the anomalies  ####

fig, ax = plt.subplots(figsize = (15, 4))
ax.plot(results['PJME_MW'],markersize = 5, linewidth = 2.5, linestyle = '-', 
        label = 'Actual value')
ax.scatter(results[results['dynamic_anomaly'] == True].index,results[results['dynamic_anomaly'] == True]['PJME_MW'], 
        color='red',label = 'Anomaly')
ax.set_xlabel('Date', fontsize = 14)
ax.set_title("dynamic anomalies", fontsize = 20)
ax.legend()
plt.show()




In [ ]:
#=================================================-
#### Slide 29: Add anomaly points  ####

add_anomaly = pjm_energy_datewise.reset_index()
add_anomaly.loc[add_anomaly['Date'] == datetime.date(2018,5,31),'PJME_MW'] = 52000
add_anomaly.loc[add_anomaly['Date'] == datetime.date(2018,2,20),'PJME_MW'] = 50000

add_anomaly = add_anomaly.set_index(['Date'])
arima = ARIMA(add_anomaly, order = (5, 1, 0), freq = 'D')
arima_fit = arima.fit()
predicted = arima_fit.predict(typ='levels')




In [ ]:
#=================================================-
#### Slide 30: Calculate error and dynamic threshold  ####

results = pd.DataFrame(add_anomaly)
results['predicted'] = predicted
results['error'] = abs(results['PJME_MW'] - results['predicted'])

results['error_windowed_mean'] = results['error'].rolling(window=7, min_periods=1).mean()
results['error_windowed_std'] = results['error'].rolling(window=7, min_periods=1).std()
results['dynamic_threshold'] = results['error_windowed_mean'] + (2*results['error_windowed_std'])
results['dynamic_anomaly'] = results['error'] >= results['dynamic_threshold']
results.head()




In [ ]:
#=================================================-
#### Slide 31: Visualize the anomalies  ####

fig, ax = plt.subplots(figsize = (15, 4))
ax.plot(results['PJME_MW'],markersize = 5, linewidth = 2.5, linestyle = '-', 
        label = 'Actual value')
ax.scatter(results[results['dynamic_anomaly'] == True].index,results[results['dynamic_anomaly'] == True]['PJME_MW'], 
        color='red',label = 'Anomaly')
ax.set_xlabel('Date', fontsize = 14)
ax.set_title("dynamic anomalies", fontsize = 20)
ax.legend()
plt.show()




In [ ]:
#=================================================-
#### Slide 34: Exercise 1  ####




#######################################################
####  CONGRATULATIONS ON COMPLETING THIS MODULE!   ####
#######################################################
